In [1]:
import numpy as np
import json

import nltk
from nltk.corpus import stopwords
import string

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/fabio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/fabio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
pad_char = 0
start_char=1
oov_char=2
index_from=3

In [3]:
def get_stopwords(): 
    punctuation = [c for c in string.punctuation]
    punctuation += [',','.','-', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}', '’', '”', '“', '``', "''", '–']
    stop_words = set(stopwords.words('italian'))
    stop_words.update(punctuation)
    return stop_words

In [4]:
assert len(get_stopwords()) > 0

In [5]:
def get_word_index(path='data_dirigenti_word_index.json'):
    with open(path, 'r') as f: 
        word_index = json.load(f)
    return word_index

In [6]:
word_index = get_word_index()
assert len(word_index) > 0

In [7]:
def get_most_common_words(path='data_dirigenti_most_common.json'):
    with open(path, 'r') as f:
        most_commons = json.load(f)
    return most_commons

In [8]:
def select_indexes_(x,y, idx_words):
    x_new = []
    y_new = []
    for sample,label in zip(x, y):
        sequence = [ w if w in idx_words else oov_char for w in sample]
        if not np.all(sequence == oov_char):
            x_new.append(sequence)
            y_new.append(label)
    return x_new, y_new

In [9]:
def load_data(path='data_dirigenti.npz', num_words=None, skip_top=0, 
              remove_stopwords=False, seed=11235):
    np.random.seed(seed)
    
    #add download path from a remote
    
    #load most_common words
    most_common = get_most_common_words()
    word_index = get_word_index()
    stop_words = get_stopwords()
    
    if remove_stopwords:
        most_common = [(k,v) for k,v in most_common if k not in stop_words]
    
    if skip_top > 0:
        most_common = most_common[skip_top:]
        
    #take only the top k words
    if num_words:
        most_common=most_common[:num_words]
    

    most_common_selected = set([k for k,v in most_common])
    
    index_word_selected = { word_index[w]:w for w in most_common_selected}
    
    with np.load(path) as loaded:
        x_train, y_train = loaded['x_train'], loaded['y_train']
        x_val, y_val = loaded['x_val'], loaded['y_val']
        x_test, y_test = loaded['x_test'], loaded['y_test']  
    
    x_train_new, y_train_new = select_indexes_(x_train, y_train, index_word_selected)
    x_val_new, y_val_new = select_indexes_(x_val, y_val, index_word_selected)
    x_test_new, y_test_new = select_indexes_(x_test, y_test, index_word_selected)
    
    return (np.array(x_train_new), np.array(y_train_new)), (np.array(x_val_new), np.array(y_val_new)), (np.array(x_test_new), np.array(y_test_new))


In [10]:
(x_train_new, y_train_new), (x_val_new, y_val_new), (x_test_new, y_test_new) = load_data(num_words=10000, remove_stopwords=False)

In [11]:
x_train_new.shape

(99390,)

In [12]:
word_index = get_word_index()
index_word = {v:k for k,v in word_index.items()}

In [13]:
for i in range(10):
    obj = [index_word[i] if i in index_word else '?' for i in x_train_new[i]]
    print(' '.join(obj))

d.p.g.r . n° 33/r del 26/07/2011 e s.m.i . sig . ? luca - appostamento fisso di caccia a minuta selvaggina fascicolo ? . autorizzazione al trasferimento di titolarità .
approvazione schema di convenzione tra la regione toscana e artigiancredito toscano scrl , relativo alla costituzione e gestione del fondo previsto dalla misura 1.6 , azione 1.6.2 `` creazione di pmi da parte di donne '' per le imprese del settore dell artigianato . impegno di spesa .
l.r.3/94 autorizzazione nuovo appostamento fisso di caccia ? alla minuta selvaggina . ? fabrizio .
missione a bruxelles nei giorni 27 e 28 marzo 2003 del dirigente claudio del re - autorizzazione e impegno di spesa
l.r . 1/06 ; delib . cr 3/12 ; delib . gr 538/13 e s.m . piano regionale agricolo forestale ( praf ) misura a.2.9 , azione a ) - annualità 2013 . liquidazione del contributo a favore a favore della pro loco santa croce sull ’ arno per la realizzazione della manifestazione “ i giorni dell ’ ? ” che si terrà a ? sull ’ arno nei gi